In [ ]:
# !pip install pandas
# !pip install torch
# !pip install tqdm  # pip install ipywidgets or something

In [22]:
USING_WANDB = True  # Set to false if not Peter

if USING_WANDB:
    import wandb
    wandb.login()
    # !wandb login

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: petern0408. Use `wandb login --relogin` to force relogin


In [32]:
import dna_dataset, constants, utils, ConvModel
import torch.nn as nn, torch.optim as optim, torch
from tqdm.notebook import tqdm
import importlib, os
from datetime import datetime

importlib.reload(dna_dataset)
importlib.reload(constants)
importlib.reload(utils)
importlib.reload(ConvModel)

<module 'ConvModel' from '/home/petern/Documents-WSL/dna-ml-model/ConvModel.py'>

Unzip the datafile

In [3]:
!unzip $constants.DATA_ZIP_FILE

Archive:  Files.zip
replace Files/accessible.fasta? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
full_dataset = dna_dataset.DNADataset(constants.ACCESSIBLE_FILE, constants.NOT_ACCESSIBLE_FILE)

In [12]:
# ensure the DNADataset is loaded properly
print('total sequences', len(full_dataset.sequences))
print('num accessible', full_dataset.accessible_count)
print('num not accessible', full_dataset.not_accessible_count)
i = 0
print(f"example entry {i}")
item = full_dataset[i]
print("label", item['labels'])
# print(item['sequences'])

# ensure dataset was shuffled properly
# check that not all the accessible labels are at the front
for i in range(full_dataset.accessible_count):
    if full_dataset[i]['labels'] != constants.ACCESSIBLE_LABEL:
        print('shuffled')
        break

total sequences 525688
num accessible 47239
num not accessible 478449
example entry 0
label 0
shuffled


In [8]:
# Split dataset
full_size = len(full_dataset)
val_size = round(constants.VALIDATION_SPLIT * full_size)
train_size = full_size - val_size

train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

print(len(train_dataset), len(val_dataset))
assert(len(train_dataset) + len(val_dataset) == full_size)

446835 78853


In [19]:
# insert torch model here, that takes sequence as input and output a label 0 or 1
model = ConvModel.CNNModel(kernel_size=2, embed_dim=4)

In [20]:
# parameters
epochs = constants.EPOCHS
batch_size = constants.BATCH_SIZE
n_eval = constants.N_EVAL
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

In [21]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=True
)

In [28]:
# TRAINING LOOP
if USING_WANDB:
    wandb.init(
        project="dna_ml_model",
        # name=f"experiment_{run}"
        config = {
            "architecture": model.__class__.__name__,
            "epochs": constants.EPOCHS,
            "learning_rate": constants.LEARNING_RATE,
            "notes": "none"
    })


step = 0

for epoch in range(epochs):
    print(f"Epoch {epoch + 1} of {epochs}")

    for batch in tqdm(train_loader):  # show the times for each batch
        # Forward propagate
        samples, labels = batch["sequences"].to(device), batch['labels'].to(device)

        outputs = model(samples)

        labels = labels.reshape(-1,1).float()
        # Backpropagation and gradient descent
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()  # reset gradients before next iteration


        # Periodically evaluate our model + log to Tensorboard
        if step % n_eval == 0:
            # Compute training loss and accuracy.
            # Log the results to Tensorboard.
            with torch.no_grad():
                # Compute validation loss and accuracy.
                accuracy = utils.compute_accuracy(outputs, labels)
                val_loss, val_accuracy = utils.evaluate(val_loader, model, loss_fn, device)

                wandb.log({"Train Loss": loss,
                           "Train Acc": accuracy,
                           "Val Loss": val_loss,
                           "Val Acc": val_accuracy,
                           "Epoch": epoch
                })

                # writer.add_scalar('Training Loss', loss, epoch)
                # writer.add_scalar('Training Accuracy', accuracy, epoch)
                # writer.add_scalar('Validation Loss', val_loss, epoch)
                # writer.add_scalar('Validation Accuracy', val_accuracy, epoch)

        step += 1

    print()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Epoch 1 of 10


  0%|          | 0/4469 [00:00<?, ?it/s]

KeyboardInterrupt: 

Save Model

In [29]:
# Create pretrained directory if not yet created
if not os.path.isdir(constants.PRETRAINED_DIR):
    os.mkdir(constants.PRETRAINED_DIR)

now = datetime.now()
datetime_str = now.strftime("%m-%d-%H-%M-%S")
model_save_path = os.path.join(
    constants.PRETRAINED_DIR,
    f'{datetime_str}-{model.__class__.__name__}-model-{constants.EPOCHS}-epochs.pt'
)
print('model_save_path', model_save_path)
ConvModel.save_CNNModel(model_save_path, model)
print(f"model saved at {datetime_str}")

model_save_path pretrained/12-09-15-51-24-CNNModel-model-CNNModel-10-epochs.pt
model saved at 12-09-15-51-24


Load Model

In [38]:
model_save_path = "pretrained/12-09-15-51-24-CNNModel-model-CNNModel-10-epochs.pt"
model = ConvModel.load_CNNModel(model_save_path)

Inference

In [41]:
total_predictions = 0
total_correct = 0

for batch in tqdm(val_loader):
    val_samples, val_labels = batch['sequences'].to(device), batch['labels'].to(device)

    val_outputs = model(val_samples)
    print(torch.round(val_outputs))
    val_labels = val_labels.reshape(-1, 1).float()

    val_loss = loss_fn(val_outputs, val_labels).item()  # change tensor to single val
    
    total_correct += (torch.round(val_outputs) == val_labels).sum().item()

    total_predictions += len(val_outputs)

val_accuracy = total_correct / total_predictions
print(f"Final Validation Accuracy: {val_accuracy * 100}%")

  0%|          | 0/789 [00:00<?, ?it/s]

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
      

Inference on Test File